In [ ]:
!pip install sparknlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=86f70d88fe502df8aeef56b60aa1276da3a0e9f3e74e03e5bbbd56cd36615fb8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Import các thư viện cần thiết
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, NGram, CountVectorizer, IDF, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Khởi tạo SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Đọc dữ liệu từ file CSV
df = spark.read.option("multiLine", True).option("header", True).option("escape", "\"").csv("Amazon_product_review.csv").na.drop()

# Hiển thị 10 dòng đầu tiên
df.show(10)

# Đếm số lượng mẫu của mỗi nhãn
sentiment_count_result = df.groupBy("Sentiment").count()
sentiment_count_result.show()

# Chia dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%)
data = df.randomSplit([0.8, 0.2])
train_set = data[0]
test_set = data[1]

# Hiển thị số lượng dòng trong tập huấn luyện và tập kiểm tra
train_count = train_set.count()
test_count = test_set.count()
print("Số lượng dòng trong tập huấn luyện:", train_count)
print("Số lượng dòng trong tập kiểm tra:", test_count)

# Đếm số lượng mẫu của mỗi nhãn trong tập huấn luyện và tập kiểm tra
sentiment_count_train = train_set.groupBy("Sentiment").count()
sentiment_count_train.show()

sentiment_count_test = test_set.groupBy("Sentiment").count()
sentiment_count_test.show()

# Tạo Pipeline
def build_ngrams_wocs(inputCol=["short_review", "Sentiment"], n=3):
    tokenizer = Tokenizer(inputCol="short_review", outputCol="words")

    ngrams = [NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i)) for i in range(1, n + 1)]

    cv = [CountVectorizer(inputCol="{0}_grams".format(i), outputCol="{0}_tf".format(i)) for i in range(1, n + 1)]
``
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    assembler = VectorAssembler(inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)], outputCol="features")

    label_stringIdx = StringIndexer(inputCol=inputCol[1], outputCol="label")

    rfc = RandomForestClassifier(numTrees=3, maxDepth=5)

    return Pipeline(stages=[tokenizer] + ngrams + cv + idf + [assembler, label_stringIdx, rfc])

# Tạo Pipeline
trigramwocs_pipeline = build_ngrams_wocs()

# Fit Pipeline trên tập huấn luyện
trigramwocs_pipeline_fit = trigramwocs_pipeline.fit(train_set)

# Dự đoán trên tập kiểm tra
predictions = trigramwocs_pipeline_fit.transform(test_set)

# Đánh giá hiệu suất của mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)



+--------------------+---------+
|        short_review|Sentiment|
+--------------------+---------+
|       Does not work| negative|
|This is a great w...| positive|
|It works great so...| positive|
|This product was ...| positive|
|it works but it h...| positive|
|Excellent product...| positive|
|This product work...| positive|
|The unit sort of ...| negative|
|I fly in the far ...| positive|
|good sound, looks...| positive|
+--------------------+---------+
only showing top 10 rows

+---------+-----+
|Sentiment|count|
+---------+-----+
| positive|15216|
| negative| 4783|
+---------+-----+

Số lượng dòng trong tập huấn luyện: 15955
Số lượng dòng trong tập kiểm tra: 4044
+---------+-----+
|Sentiment|count|
+---------+-----+
| positive|12180|
| negative| 3775|
+---------+-----+

+---------+-----+
|Sentiment|count|
+---------+-----+
| positive| 3036|
| negative| 1008|
+---------+-----+

Accuracy: 0.7519782393669634
